In [3]:
import pandas as pd
import os

In [16]:
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')
import plotly.express as px
import plotly.graph_objects as go


In [17]:
at_prim = pd.read_csv('../data/psico_atprimaria.csv')

In [18]:
at_prim.sample(3)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10,resumen
89325,2019,Aragon,Hombre,- 18000,00-14,975,2,P06,Trastornos del sueño,F30-39,animo/afectivos
103508,2019,Valencia,Hombre,desconocida,65 +,0,10,P22+23,Signos/síntomas de comportamiento del niño/ado...,F90-F98,aparecen en niñez
33309,2017,Asturias,Mujer,18000-99999,00-14,0,3,P02,Estrés agudo,F40-49,nuróticos/somatomorfos


In [19]:
# 'Trastornos de la ansiedad/estado de ansiedad' -> P74

In [20]:
at_prim.Edad.unique()

array(['00-14', '15-34', '35-64', '65 +'], dtype=object)

In [21]:
edad = {'00-14':0, '15-34':15, '35-64':35, '65 +':65}

In [22]:
at_prim.Edad = at_prim.Edad.map(edad)

In [23]:
ag_cei10 = at_prim.groupby(['Año','resumen']).agg({"Casos":"sum"}).reset_index()

In [41]:
fig_atprim = px.line(ag_cei10, x="Año", y="Casos",color = 'resumen')
fig_atprim.show()

In [42]:
fig_atprim.write_html('../output/fig_atprim.html')

In [16]:
at_prim.sample()

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10,title
22090,2016,Murcia,Hombre,18000-99999,0,0,14,P03,Sensación/sentimientos depresivos,F40-49,"Trastornos neuróticos, trastornos relacionados..."


In [17]:
casos = at_prim.groupby(['Año','CCAA','Sexo','Nivel_Renta','cei10']).agg({'Casos':'sum'}).reset_index()

In [18]:
casos_renta = casos.groupby(['Año','Nivel_Renta']).agg({'Casos':'sum'}).reset_index()

In [19]:
casos_renta.sample(3)

,Año,Nivel_Renta,Casos
9,2017,muy_baja,1368371
10,2018,+ 100000,31652
5,2017,+ 100000,30119


In [20]:
casos_renta.Nivel_Renta.value_counts()

+ 100000       4
- 18000        4
18000-99999    4
desconocida    4
muy_baja       4
Name: Nivel_Renta, dtype: int64

In [21]:
pob_act = pd.read_csv('../data/pob_activa_acum.csv')
pob_act.sample(3)

,sexo,cod_ca,edad,year,miles
669,Mujeres,9,De 45 a 54 años,2017,419.650
790,Mujeres,14,De 45 a 54 años,2018,64.875
525,Mujeres,3,De 45 a 54 años,2017,53.650


In [22]:
pob_act['tot'] = pob_act.miles * 1000 

In [23]:
activos = pob_act.groupby(['year']).agg({'tot':'sum'}).astype('int64').reset_index()

In [24]:
activos.sample(3)

,year,tot
1,2017,18824775
3,2019,19779375
2,2018,19327125


In [25]:
dic_activos={}
for i,r in activos.iterrows():
    dic_activos[int(r.year)] = int(r.tot)
dic_activos

{2016: 18341600, 2017: 18824775, 2018: 19327125, 2019: 19779375}

In [25]:
# fig1 = px.line(casos_renta, x="Año", y="Casos",color = 'Nivel_Renta')
# fig1.show()

In [27]:
nivel_renta = at_prim.groupby(['Nivel_Renta','cei10']).agg({'Casos':'sum'}).reset_index()

In [28]:
rent_level = {'+ 100000':4, '- 18000':2, '18000-99999':3, 'desconocida':0, 'muy_baja':1}

In [29]:
# nivel_renta.Nivel_Renta.unique()
# renta_dic = {'+ 100000':5, '- 18000':2, '18000-99999':3, 'desconocida':0, 'muy_baja':1}
# nivel_renta.Nivel_Renta = nivel_renta.Nivel_Renta.map(renta_dic)

In [30]:
nivel_renta["renta"] = nivel_renta.Nivel_Renta.map(rent_level)

In [31]:
fig2 = px.histogram(nivel_renta, x="Nivel_Renta", y="Casos",color = 'cei10') 
fig2.show()

In [32]:
nivel_renta_sex = at_prim.groupby(['Nivel_Renta','Sexo']).agg({'Casos':'sum'}).reset_index()
fig3 = px.histogram(nivel_renta_sex, x="Nivel_Renta", y="Casos",color = 'Sexo') 
fig3.show()

In [26]:
at_prim.sample(3)

,Año,CCAA,Sexo,Nivel_Renta,Edad,Casos,CodCA,cod_salud,problema_salud,cei10,resumen
58568,2018,Andalucia,Hombre,muy_baja,15,0,1,P05,Sentimientos/conducta senil,F00-F09,orgánicos/sintomáticos
42965,2017,Cataluña,Hombre,18000-99999,65,0,9,P24,Problemas específicos del aprendizaje,F80-89,desarrollo psicológico
37213,2017,Canarias,Mujer,muy_baja,15,0,5,P77,Suicidio/intento de suicidio,F30-39,animo/afectivos


In [39]:
at_prim.groupby(['Nivel_Renta','cei10','resumen']).agg({'Casos':'sum'}).reset_index().sort_values(['Nivel_Renta','Casos'],ascending=False)

,Nivel_Renta,cei10,resumen,Casos
48,muy_baja,F40-49,nuróticos/somatomorfos,1932467
47,muy_baja,F30-39,animo/afectivos,1252752
45,muy_baja,F10-F19,consumo psicotrópicos,1128190
46,muy_baja,F20-29,esquizotípicos/delirantes,360826
50,muy_baja,F60-69,T. de la personalidad/comportamiento,147988
44,muy_baja,F00-F09,orgánicos/sintomáticos,138071
53,muy_baja,F90-F98,aparecen en niñez,86182
54,muy_baja,F99,sin espcificar,60744
51,muy_baja,F70-79,deficiencia psíquica,60294
52,muy_baja,F80-89,desarrollo psicológico,49041
